<a href="https://colab.research.google.com/github/sumaaithal/PySpark_30Days_Challenge/blob/main/pyspark35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,390 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,713 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [23.8 kB]
Get:13 https://ppa.launchpadcontent.net/ubuntugis

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [17]:
from pyspark.sql.functions import current_date, datediff,col, months_between, round,lit, to_date

In [5]:
data = [("1","2019-07-01"),("2","2019-06-24"),("3","2019-08-24")]

df = spark.createDataFrame(data=data,schema=["id","date"])
df.show()

+---+----------+
| id|      date|
+---+----------+
|  1|2019-07-01|
|  2|2019-06-24|
|  3|2019-08-24|
+---+----------+



In [7]:
df.select(col("date"),
          current_date().alias("current_date"),
          datediff(current_date(), col("date")).alias("date_difference")).show()

+----------+------------+---------------+
|      date|current_date|date_difference|
+----------+------------+---------------+
|2019-07-01|  2024-01-28|           1672|
|2019-06-24|  2024-01-28|           1679|
|2019-08-24|  2024-01-28|           1618|
+----------+------------+---------------+



In [16]:
df.select(current_date().alias("current_date"),
          datediff(current_date(), col("date")).alias("date_difference"),
          round( datediff(current_date(), col("date"))  ,2).alias("round_date_difference"),
          (months_between(current_date(),col("date"))/lit(12)).alias("years"),
          round((months_between(current_date(),col("date"))/lit(12)),2).alias("rounded_years")).show()

+------------+---------------+---------------------+------------------+-------------+
|current_date|date_difference|round_date_difference|             years|rounded_years|
+------------+---------------+---------------------+------------------+-------------+
|  2024-01-28|           1672|                 1672|4.5725806449999995|         4.57|
|  2024-01-28|           1679|                 1679| 4.594086021666667|         4.59|
|  2024-01-28|           1618|                 1618|4.4274193550000005|         4.43|
+------------+---------------+---------------------+------------------+-------------+



In [18]:
data2 = [("1","07-01-2019"),("2","06-24-2019"),("3","08-24-2019")]

df2 = spark.createDataFrame(data=data2, schema=['id',"date"])

df2.show()

+---+----------+
| id|      date|
+---+----------+
|  1|07-01-2019|
|  2|06-24-2019|
|  3|08-24-2019|
+---+----------+



In [20]:
df2.select(to_date(col("date"),'MM-dd-yyyy').alias("date"),
           current_date().alias("current-date")).show()

+----------+------------+
|      date|current-date|
+----------+------------+
|2019-07-01|  2024-01-28|
|2019-06-24|  2024-01-28|
|2019-08-24|  2024-01-28|
+----------+------------+



In [21]:
spark.sql("select round(months_between('2019-01-09', current_date())/12,2) as years").show()

+-----+
|years|
+-----+
|-5.05|
+-----+

